In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg

In [ ]:
os.listdir('../input/rsna-2022-cervical-spine-fracture-detection')

# Lets see how the data looks like

In [ ]:
data=pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/train.csv')
data.head()

In [ ]:
## Taking a small amount of the training data for quick experiment results.

path='../input/rsna-2022-cervical-spine-fracture-detection/train_images'
li=os.listdir('../input/rsna-2022-cervical-spine-fracture-detection/train_images')
img_paths=[]
for i in range(0,5):
    img_paths.extend([os.path.join(path,li[i],j) for j in os.listdir(os.path.join(path,li[i]))])
len(img_paths)

In [ ]:
# Importing some important libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler


import warnings
warnings.filterwarnings('ignore')

# Defining the Resnet Network

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, filters, subsample=False):
        super().__init__()
        
        if(subsample == True):
            self.conv1 = nn.Conv2d(int(filters/2), filters, kernel_size=3, stride=2, padding=1, bias=False)
        else:
            self.conv1 = nn.Conv2d(int(filters), filters, kernel_size=3, stride=1, padding=1, bias=False)
            
        self.bn1   = nn.BatchNorm2d(filters, track_running_stats=True)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(filters, track_running_stats=True)
        self.relu2 = nn.ReLU()

        self.downsample = nn.AvgPool2d(kernel_size=1, stride=2)

        for layer in self.modules():
            if isinstance(layer, nn.Conv2d):
                nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(layer, nn.BatchNorm2d):
                nn.init.constant_(layer.weight, 1)
                nn.init.constant_(layer.bias, 0)        
    
    def forward(self, x):
        z = self.conv1(x)
        z = self.bn1(z)
        z = self.relu1(z)
        
        z = self.conv2(z)
        z = self.bn2(z)

        z = self.relu2(z)
        
        return z
    


class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convIn = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bnorm   = nn.BatchNorm2d(16, track_running_stats=True)
        self.relu   = nn.ReLU()
        
        self.part1 = nn.ModuleList([BasicBlock(16, subsample=False), BasicBlock(16, subsample=False), BasicBlock(16, subsample=False)])

        self.part2a = BasicBlock(32, subsample=True)
        self.part2b = nn.ModuleList([BasicBlock(32, subsample=False), BasicBlock(32, subsample=False)])

        self.part3a = BasicBlock(64, subsample=True)
        self.part3b = nn.ModuleList([BasicBlock(64, subsample=False), BasicBlock(64, subsample=False)])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.out   = nn.Linear(64, 8, bias=True)

        self.Softmax = nn.Softmax(dim=-1)

        
        # Initilise weights
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal(layer.weight)
                layer.bias.data.zero_()      
        
        
    def forward(self, x):     
        x = self.convIn(x)
        x = self.bnorm(x)
        
        x = self.relu(x)
        for layer in self.part1: 
            x = layer(x)
        
        x = self.part2a(x)
        for layer in self.part2b: 
            x = layer(x)
        
        x = self.part3a(x)
        for layer in self.part3b: 
            x = layer(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.out(x)
        return self.Softmax(x)


In [ ]:
# Lets define an instance of the Resnet Network and see whether it performs.

import numpy as np
a=ResNet(3)
b=torch.randn([1,3,32,32])
a(b)

# Defining the Data Loading Class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import pydicom as dicom
import cv2
import os

def load_dicom(path, size = 512):
    
    img=dicom.dcmread(path)
    img.PhotometricInterpretation = 'YBR_FULL'
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    #data=data.astype(np.uint8)
    return cv2.cvtColor(cv2.resize(data,(32,32)), cv2.COLOR_GRAY2RGB).transpose([2,0,1]).astype(np.float32)

class dcm_data(Dataset):
    def __init__(self,image_paths,data):
        self.image_paths=image_paths
        self.data=data
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self,idx):
        image_filepath=self.image_paths[idx]
        image=load_dicom(image_filepath)
        li_tensor=[]
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['patient_overall'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C1'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C2'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C3'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C4'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C5'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C6'].values))
        li_tensor.append(int(self.data[self.data['StudyInstanceUID']==image_filepath.split('/')[-2]]['C7'].values))
        return image,torch.Tensor(li_tensor)


def get_data_loaders(data_dir,data,
                     batch_size,
                     shuffle=True,
                     num_workers=4,
                     pin_memory=False):

    

    train_dataset=dcm_data(data_dir,data)
    test_dataset=dcm_data(data_dir,data)
    
    # Create loader objects
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory
    )
          
    return (train_loader, test_loader)

# Defining the Training and evaluation functions

In [ ]:
import numpy as np
import pandas as pd
import torch

def evaluate(model, data_loader, device):
    
    y_true = np.array([[]], dtype=np.int)
    y_pred = np.array([[]], dtype=np.int)
    
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            print(y_true)
            print(labels.cpu())
            y_true = np.concatenate((y_true, labels.cpu()))
            y_pred = np.concatenate((y_pred, predicted.cpu()))
    
    error = np.sum(y_pred != y_true) / len(y_true)
    return error



def train_model(model, epochs, train_loader, test_loader, criterion, optimizer):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model.to(device)
    
    train_loss_li=[]
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        model.train()
        for i, data in enumerate(train_loader, 0):   # Do a batch iteration
            
            # get the inputs
            inputs, labels = data   
            inputs, labels = inputs.to(device), labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            # print average loss for last 3 mini-batches
            total_loss += loss.item()
            if i % 3 == 2:
                print('Epoch : {}, Batch : {}, Loss : {}'.format((epoch + 1, i + 1, total_loss / 3)))
                     
                train_loss_li.append(total_loss/3)
                total_loss = 0.0
        
        # Record metrics
        model.eval()
        train_loss = loss.item()    
    
    print('Finished Training with train_loss = {}'.format(train_loss))
    return train_loss_li

In [ ]:
# lets use the data loader function and see if it outputs data. If it dosent work, restart the kernel and clear outputs. Run the notebook again and you will see that it works.

train_ss,test_hhs=get_data_loaders(img_paths,data,128,shuffle=True,num_workers=4,pin_memory=False)
data_load=iter(train_ss)
images,labels=data_load.next()
print(labels)

# Defining Training Hyperparameters

In [ ]:
# TRAINING PARAMETERS
# -------------------------
# This epoch can be changed, however for quick experiments, we kept the epoch value at 10.
epochs = 10

lr = 0.1
momentum = 0.9
weight_decay = 0.0001 
gamma = 0.1

# Training the Model

In [ ]:
# TRAIN PLAIN NETs

# n determines network size as described in paper
# where total number of layers is (6*n)+2


train_loader, test_loader = data_loaders(img_paths,data,
                                                     512,
                                                     shuffle=True,
                                                     num_workers=4,
                                                     pin_memory=True)

model = ResNet()
    
criterion = torch.nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
loss_list=train_model(model, epochs, train_loader, test_loader, criterion, optimizer)

In [ ]:
plt.plot(np.array(loss_list), linestyle = 'dotted')
plt.xlabel('epoch number')
plt.ylabel('BCELoss')
plt.show()